In [1]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
import json
from datetime import datetime 
from alpaca_trade_api.rest import REST, TimeFrame
import time
from time import sleep
import datetime
%matplotlib inline

In [2]:
# Load .env enviroment variables
load_dotenv()
opensea_api_key = os.getenv("opensea_api_key")
opensea_api_key

'9796bf59fe674cae9688a1a9bb422338'

In [4]:
opensea_collections_url = "https://api.opensea.io/api/v1/collections"
opensea_collection_stats_url = "https://api.opensea.io/api/v1/collection/doodles-official/stats"

In [11]:
def get_opensea_api_data(url):
    start_date = datetime.datetime(2022, 4, 10)
    end_date = datetime.datetime(2022, 4, 11)

    headers = {
        "Accept": "application/json",
        "X-API-KEY": opensea_api_key
    }
    
    query = { 
        "only_opensea": "true", 
        "occurred_before": end_date,
        "occurred_after": start_date,
        "event_type":"successful"
    }
    
    
    response_data = requests.request("GET", url, headers=headers, params=query)
    
    #response_data = requests.get(url)
    #response_data.
    #response_data = requests.get(url).json()
   # print(json.dumps(response_data, indent=2))
    #print(response_data)
    return response_data

In [12]:
#get_opensea_api_data(opensea_collections_url)
#get_opensea_api_data(opensea_collection_stats_url)

response_data = get_opensea_api_data("https://api.opensea.io/api/v1/events")
json=response_data.json()
print(json)

{'next': 'LWV2ZW50X3RpbWVzdGFtcD0yMDIyLTA0LTEwKzIzJTNBNTklM0E1MyYtcGs9NDg5ODc3NjA1NQ==', 'previous': None, 'asset_events': [{'approved_account': None, 'asset': {'id': 381767439, 'num_sales': 2, 'background_color': None, 'image_url': 'https://lh3.googleusercontent.com/DlTfmHnPGO3cJqRVG1_B0xVU2GLuZ3Hhi7R8qaDkf1xqSQtZCCC8cWV8xGiVQoD26Rh2B6H1ro_u2hLFnlPz3bK3EhjtGFVkL_fTigU', 'image_preview_url': 'https://lh3.googleusercontent.com/DlTfmHnPGO3cJqRVG1_B0xVU2GLuZ3Hhi7R8qaDkf1xqSQtZCCC8cWV8xGiVQoD26Rh2B6H1ro_u2hLFnlPz3bK3EhjtGFVkL_fTigU=s250', 'image_thumbnail_url': 'https://lh3.googleusercontent.com/DlTfmHnPGO3cJqRVG1_B0xVU2GLuZ3Hhi7R8qaDkf1xqSQtZCCC8cWV8xGiVQoD26Rh2B6H1ro_u2hLFnlPz3bK3EhjtGFVkL_fTigU=s128', 'image_original_url': 'ipfs://QmX8ERT5LQaZZR4cCdM1k9ownHsib5Kh1v5fBGaNMFdNtQ/1201.png', 'animation_url': None, 'animation_original_url': None, 'name': 'sketchy mfer #1201', 'description': 'sketchy mfers by jellopeas, sodasoondae, and zhoug', 'external_link': None, 'asset_contract': {'addre

In [15]:
def get_events(start_date, end_date, cursor='', event_type='successful', **kwargs):
    url = "https://api.opensea.io/api/v1/events"
    
    query = { 
        "only_opensea": "false", 
        "occurred_before": end_date,
        "occurred_after": start_date,
        "event_type": event_type,
        "cursor": cursor,
         **kwargs
    }

    headers = {
        "Accept": "application/json",
        "X-API-KEY": opensea_api_key
    }
    response = requests.request("GET", url, headers=headers, params=query)

    return response.json()

In [41]:
def parse_event(event):
    record = {}
    asset = event.get('asset')
    if asset == None:
        return None # if there's no asset that means it's not a single NFT transaction so skip this item

    #collection
    record['collection_slug'] = asset['collection']['slug']
    record['collection_name'] = asset['collection']['name']
    record['collection_url'] = "https://opensea.io/collection/" + asset['collection']['slug']

    #asset
    record['asset_id'] = asset['id']
    record['asset_name'] = asset['name']
    record['asset_description'] = asset['description']
    record['asset_contract_date'] = asset['asset_contract']['created_date']
    record['asset_url'] = asset['permalink']
    record['asset_img_url'] = asset['image_url']

    #event
    record['event_id'] = event['id']
    record['event_time'] = event.get('created_date')
    record['event_auction_type'] = event.get('auction_type')
    record['event_contract_address'] = event.get('contract_address')
    record['event_quantity'] = event.get('quantity')
    record['event_payment_symbol'] =  None if event.get('payment_token') == None else event.get('payment_token').get('symbol')

    decimals = 18
    if event.get('payment_token') != None:
        decimals = event.get('payment_token').get('decimals')

    price_str = event['total_price']

    try: 
        if len(price_str) < decimals:
            price_str =  "0." + (decimals-len(price_str)) * "0" + price_str
            record['event_total_price'] = float(price_str)
        else:
            record['event_total_price'] = float(price_str[:-decimals] + "." + price_str[len(price_str)-decimals:])
    except:
        print(event)

    return record

In [42]:
def fetch_all_events(start_date, end_date, pause=1, **kwargs):
    result = list()
    next = ''
    fetch = True

    print(f"Fetching events between {start_date} and {end_date}")
    while fetch:
        response = get_events(int(start_date.timestamp()), int(end_date.timestamp()), cursor=next, **kwargs)

        for event in response['asset_events']:
            cleaned_event = parse_event(event)
            
            if cleaned_event != None:
                result.append(cleaned_event)

        if response['next'] is None:
            fetch = False
        else:
            next = response['next']

        sleep(pause)

    return result

In [ ]:


start_date = datetime.datetime(2022, 2, 10)
end_date = datetime.datetime(2022, 4, 10)

result = fetch_all_events(x,x)
result

Fetching events between 2020-05-17 00:00:00 and 2020-05-17 00:00:00
{'approved_account': None, 'asset': {'id': 367331547, 'num_sales': 0, 'background_color': None, 'image_url': 'https://lh3.googleusercontent.com/PE32wRxUTc1Lt2A3kG-uoeci0MGHjiU0J98IXcFTPTQXNTD5rfCSIWSQmv4QiOOzRQfakQx3w9OlAZ5D_qclM5sUhYhrU4_4AxSJ', 'image_preview_url': 'https://lh3.googleusercontent.com/PE32wRxUTc1Lt2A3kG-uoeci0MGHjiU0J98IXcFTPTQXNTD5rfCSIWSQmv4QiOOzRQfakQx3w9OlAZ5D_qclM5sUhYhrU4_4AxSJ=s250', 'image_thumbnail_url': 'https://lh3.googleusercontent.com/PE32wRxUTc1Lt2A3kG-uoeci0MGHjiU0J98IXcFTPTQXNTD5rfCSIWSQmv4QiOOzRQfakQx3w9OlAZ5D_qclM5sUhYhrU4_4AxSJ=s128', 'image_original_url': None, 'animation_url': None, 'animation_original_url': None, 'name': '#1445', 'description': None, 'external_link': None, 'asset_contract': {'address': '0x2953399124f0cbb46d2cbacd8a89cf0599974963', 'asset_contract_type': 'semi-fungible', 'created_date': '2021-08-11T02:56:18.366467', 'name': 'OpenSea Collections', 'nft_version': Non